# Preparations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.2 MB/s eta 0:00:00


In [ ]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


## Datasets

In [ ]:
import os
HOME = os.getcwd()
print(HOME)
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from roboflow import Roboflow
rf = Roboflow(api_key="QuI0m8QKzWrmYdKhcfqF")
project = rf.workspace("deep-learning-hbzyf").project("ie_dp")
version = project.version(11)
dataset = version.download("yolov8")

/content/datasets
/content/datasets/datasets
loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.39, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to IE_DP-11 in yolov8:: 100%|██████████| 676/676 [00:00<00:00, 6136.03it/s]


# Fine tuning

In [ ]:
from ultralytics import YOLO

In [ ]:
# creating the yolo file
import yaml
import os

# path
dataset_path = '/content/datasets/datasets/IE_DP-11'

# filepath
data_yaml_path = os.path.join(dataset_path, 'data.yaml')
train_images_path = os.path.join(dataset_path, 'train/images')
val_images_path = os.path.join(dataset_path, 'valid/images')
test_images_path = os.path.join(dataset_path, 'test/images')

# categories
num_classes = 1  # numbers
class_names = ['IE_bag']

# checking and correcting data.yaml
def check_and_modify_yaml():
    if os.path.exists(data_yaml_path):
        with open(data_yaml_path, 'r') as f:
            data = yaml.safe_load(f)
    else:
        data = {}

    data['train'] = train_images_path
    data['val'] = val_images_path

    data['nc'] = num_classes
    data['names'] = class_names

    with open(data_yaml_path, 'w') as f:
        yaml.safe_dump(data, f, default_flow_style=False)

    print(f'data.yaml has been checked and modified: {data_yaml_path}')

check_and_modify_yaml()


data.yaml has been checked and modified: /content/datasets/datasets/IE_DP-11/data.yaml


In [ ]:
!yolo task=detect \
mode=train \
model=yolov8s.pt \
data={dataset.location}/data.yaml \
epochs=50 \
imgsz=640

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/datasets/datasets/IE_DP-11/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, s

In [ ]:
!cp /content/datasets/datasets/runs/detect/train3/weights/best.pt /content/drive/MyDrive/IE/Deep_Learning/best.pt

## Testing

In [ ]:
# based on a certain image
!yolo task=detect \
mode=predict \
model='/content/drive/MyDrive/IE/Deep_Learning/best.pt' \
conf=0.25 \
source='/content/datasets/datasets/IE_DP-11/test/WechatIMG34.jpg'

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
image 1/1 /content/datasets/datasets/IE_DP-11/test/WechatIMG34.jpg: 640x480 1 IE_bag, 71.7ms
Speed: 3.3ms preprocess, 71.7ms inference, 518.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict4
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
# based on the test dataset
!yolo task=detect \
mode=predict \
model='/content/drive/MyDrive/IE/Deep_Learning/best.pt' \
conf=0.25 \
source='/content/datasets/IE_DP-9/test/images'

# Deployment

## Installing required libraries

In [ ]:
!pip install tensorflow==2.14.0
!pip install tflite_support==0.4.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 94.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.0
    Uninstalling google-auth-oauthlib-1.2.0:
      Successfully uninstalled google-auth-oauthlib-1.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling t

## Exporting the files

In [ ]:
from ultralytics import YOLO
model = YOLO('/content/drive/MyDrive/IE/Deep_Learning/best.pt')
model.export(format="tflite")

Ultralytics YOLOv8.2.39 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/IE/Deep_Learning/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (21.5 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnx2tf>1.17.5,<=1.22.3', 'sng4onnx>=1.0.1', 'onnxslim>=0.1.31', 'onnx_graphsurgeon>=0.3.26', 'onnxruntime'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 126.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 302.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 317.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 261.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 227.2 MB/s eta 

100%|██████████| 1.11M/1.11M [00:00<00:00, 53.8MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/datasets/datasets/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 37.39file/s]



ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: slimming with onnxslim 0.1.31...
ONNX: export success ✅ 6.7s, saved as '/content/drive/MyDrive/IE/Deep_Learning/best.onnx' (42.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.22.3...
TensorFlow SavedModel: export success ✅ 46.4s, saved as '/content/drive/MyDrive/IE/Deep_Learning/best_saved_model' (106.9 MB)

TensorFlow Lite: starting export with tensorflow 2.14.0...
TensorFlow Lite: export success ✅ 0.0s, saved as '/content/drive/MyDrive/IE/Deep_Learning/best_saved_model/best_float32.tflite' (42.7 MB)

Export complete (48.9s)
Results saved to /content/drive/MyDrive/IE/Deep_Learning
Predict:         yolo predict task=detect model=/content/drive/MyDrive/IE/Deep_Learning/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/IE/Deep_Learning/best_saved_model/best_float32.tflite imgsz=640 data=/content/datasets/datasets/IE_DP-11/data.yaml  
Visualize:   

'/content/drive/MyDrive/IE/Deep_Learning/best_saved_model/best_float32.tflite'